<a href="https://colab.research.google.com/github/JoshuaFZ/-/blob/main/owon_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall -y unsloth unsloth_zoo
!pip install --no-cache-dir -U \
  git+https://github.com/unslothai/unsloth.git \
  git+https://github.com/unslothai/unsloth-zoo.git \
  trl peft accelerate bitsandbytes datasets


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-i9l9yrwe
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-i9l9yrwe
  Resolved https://github.com/unslothai/unsloth.git to commit 9e75e1b7e3c5ebb28787aa8ae0bbdf40afdde8d8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-req-build-ju3d2qrw
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-req-build-ju3d2qrw
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit 502fa53afd49c26242f757844adb4796aa954388
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [3]:
import torch
from unsloth import FastLanguageModel

# 6. 配置模型参数
max_seq_length = 2048
dtype = None
load_in_4bit = True

# 7. 加载模型
model_name = "Qwen/Qwen3-0.6B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 8. 转换模型为 LoRA 模式
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Qwen3 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2026.2.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# 有问题代码，先留着
from datasets import load_dataset

# 5. 定义数据处理格式
# 将JSONL 格式转换为模型能听懂的 Prompt 格式
alpaca_prompt = """Below is an instruction that describes a task. Write a Response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # 句子结束符

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # 填充模板
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 6. 加载并处理数据集
# 确保你的文件名是 'train.jsonl'
dataset = load_dataset("json", data_files="/content/drive/MyDrive/train.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

In [5]:
# 现在调试用代码
import json
from datasets import load_dataset

# 1. 改回标准 Alpaca 模板 (使用 ### Response:)
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # 【核心修改】清洗数据：如果 output 是字典，去掉所有值为 None 的键
        if isinstance(output, dict):
            # 这一行是关键：只保留值不为 None 的字段
            clean_output = {k: v for k, v in output.items() if v is not None}
            output_str = json.dumps(clean_output, ensure_ascii=False)
        else:
            output_str = str(output)

        text = alpaca_prompt.format(instruction, input, output_str) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 加载数据
dataset = load_dataset("json", data_files="/content/drive/MyDrive/train.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

# 3. 【新增】打印一条处理后的数据，检查格式是否正确！
print("检查第一条训练数据格式：")
print(dataset["text"][0])
# 务必确认这里看到的是 {"intent": ...} (双引号)，而不是 {'intent': ...} (单引号)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

检查第一条训练数据格式：
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
你是一个工业级示波器指令解析引擎。你必须只输出 JSON，不得包含任何解释性文本。

### Input:
设置通道1的电压档位为2伏

### Response:
{"intent": "channel_scale", "channel": 1, "value": 2.0}<|im_end|>


In [ ]:
# 有问题，先保留
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# 7. 设置训练参数
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # 如果数据量很大可以设为 True 加速
    args = TrainingArguments(
        per_device_train_batch_size = 4, # 降低 Batch Size
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        # max_steps = 60,  <-- 删除这一行
        num_train_epochs = 10, # <-- 显式指定跑 10 到 15 轮
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", # 建议由 linear 改为 cosine
        seed = 3407,
        output_dir = "outputs",
    ),
)

# 8. 开始训练
trainer_stats = trainer.train()

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4, # 显存允许的话，越小更新越频繁
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 60,  <-- 删掉这个
        num_train_epochs = 15, # 【修改】直接指定跑 15 轮，确保学会格式
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=5):   0%|          | 0/60 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 60 | Num Epochs = 15 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"
wandb: Using W&B in offline mode.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,4.199800
2,4.231500
3,3.934800
4,3.721700
5,3.270900
6,2.871800
7,2.521300
8,2.054900
9,1.660000
10,1.446500


wandb: WARNING URL not available in offline run


train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,█▄▄▅▆▃▃▃▂▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▄▅▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁
train/loss,██▆▅▄▃▃▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,197475041280000.0
train/epoch,15
train/global_step,60
train/grad_norm,0.50043
train/learning_rate,0.0
train/loss,0.1644


In [17]:
FastLanguageModel.for_inference(model)

# 必须使用完全一致的 Prompt 模板
inputs = tokenizer(
[
    alpaca_prompt.format(
        "你是一个工业级示波器指令解析引擎。你必须只输出 JSON，不得包含任何解释性文本。",
        "把通道2的波形存下来",
        "", # 这里留空
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
# 解码时跳过 prompt 部分
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
你是一个工业级示波器指令解析引擎。你必须只输出 JSON，不得包含任何解释性文本。

### Input:
把通道2的波形存下来

### Response:
{"intent": "save_waveform", "channels": [2]}


In [ ]:
# 有问题，先保留
FastLanguageModel.for_inference(model) # 开启推理模式
inputs = tokenizer(
[
    alpaca_prompt.format(
        "你是一个工业级示波器指令解析引擎。你必须只输出 JSON，不得包含任何解释性文本。",
        "开启ZOOM",
        "{", # <--- 关键修改：预填充一个左大括号
    )
], return_tensors = "pt").to("cuda")


outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print(tokenizer.batch_decode(outputs))

In [ ]:
# 建议在 Drive 里建一个专门的文件夹，比如 'oscilloscope_project'
import os
save_path = "/content/drive/MyDrive/oscilloscope_project/lora_model"

if not os.path.exists(save_path):
    os.makedirs(save_path)

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"✅ 模型已安全保存到 Google Drive: {save_path}")

In [ ]:
# 1. 确保已挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. 定义 Google Drive 中的保存路径
# 建议保存在专属文件夹中，方便下载到 Ubuntu
save_directory = "/content/drive/MyDrive/oscilloscope_project/qwen_merged_hf"

print(f"正在合并模型并保存至: {save_directory} ...")

# 3. 合并并导出为 16bit 格式 (RKLLM 转换的最佳兼容格式)
model.save_pretrained_merged(
    save_directory,
    tokenizer,
    save_method = "merged_16bit",
)

print("✅ 模型合并完成！你现在可以在 Google Drive 网页端看到 qwen_merged_hf 文件夹。")
!zip -r merged_model.zip {save_directory}
#

In [ ]:
from google.colab import files

files.download('merged_model.zip')

In [ ]:
import os
import sys
import shutil
from google.colab import drive

# 1. 挂载 Google Drive
drive.mount("/content/drive")

# 2. 定义保存环境的 Google Drive 路径
lib_path = "/content/drive/MyDrive/colab_env_libs"
if not os.path.exists(lib_path):
    os.makedirs(lib_path)

# 3. 将该路径加入到 Python 的系统路径中 (优先于系统库)
if lib_path not in sys.path:
    sys.path.insert(0, lib_path)

print(f"✅ 已加载外部库路径: {lib_path}")

# 4. 安装/更新环境到 Drive
# 【关键修改】改为 False，下次检测到文件存在就会自动跳过安装，节省时间
FORCE_UPDATE = False

target_exists = os.path.exists(os.path.join(lib_path, "unsloth"))

if target_exists and not FORCE_UPDATE:
    print("✅ 检测到 Google Drive 中已存在 Unsloth 环境，跳过安装步骤！")
    print("   (如需强制更新，请将代码中的 FORCE_UPDATE 改为 True)")
else:
    if FORCE_UPDATE:
        print("🔄 正在强制更新 Google Drive 中的环境 (参考最新修复配置)...")
    else:
        print("⚡️ Drive 中未检测到环境，正在安装...")

    # 添加 --no-input 参数以禁止弹出对话框/确认提示

    # 1. 安装 unsloth_zoo
    !pip install --no-input --upgrade --no-cache-dir --no-deps --target={lib_path} git+https://github.com/unslothai/unsloth-zoo.git

    # 2. 安装 unsloth 主程序
    !pip install --no-input --upgrade --no-cache-dir --target={lib_path} "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

    # 3. 安装其他依赖
    !pip install --no-input --no-deps --target={lib_path} xformers "trl<0.9.0" peft accelerate bitsandbytes

    print("🎉 环境已成功安装/更新到 Google Drive！下次启动可直接加载。")

In [ ]:
print(f"正在从 {save_directory} 加载合并后的模型...")

# 由于是合并后的模型，可以直接加载为标准的 Hugging Face 模型
# 注意：此处使用的 FastLanguageModel.from_pretrained 是针对 Unsloth 优化过的模型加载方式
# 对于完全合并的模型，也可以使用 AutoModelForCausalLM.from_pretrained
# 但为了保持一致性，我们继续使用 FastLanguageModel
merged_model, merged_tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_directory, # 指向保存合并模型的目录
    max_seq_length = max_seq_length,
    dtype = None, # 自动检测数据类型
    load_in_4bit = False, # 合并后的模型通常不需要再进行 4bit 量化加载
)

print("✅ 合并模型加载完成！")